In [18]:
# Import the dependencies.
# Import the datetime module from the datetime library.
from datetime import datetime
import pandas as pd
import numpy as np
import requests
from citipy import citipy
import json
import subprocess
import psycopg2

In [19]:
# Import the API key.
from config import client_id, client_secret, redirect_url, base_url, oauth_url

url = "https://api.procore.com/oauth/token?grant_type=client_credentials&client_id=" + client_id + "&client_secret=" + client_secret

In [20]:
response = requests.post(url).json()
token = response.get('access_token')

In [21]:
get_projects = "https://api.procore.com/rest/v1.0/projects?company_id=7930"

projects = requests.get(get_projects, headers={"Authorization": "Bearer " + token}).json()

In [22]:
projects_df = pd.json_normalize(projects)

projects_df

,id,name,display_name,project_number,address,city,state_code,country_code,zip,county,...,custom_fields.custom_field_234819.data_type,custom_fields.custom_field_234819.value,custom_fields.custom_field_235145.data_type,custom_fields.custom_field_235145.value,custom_fields.custom_field_241562.data_type,custom_fields.custom_field_241562.value,company.id,company.name,custom_fields.custom_field_235145.value.id,custom_fields.custom_field_235145.value.label
0,1740268,1030 Fort Worth Avenue,21051 - 1030 Fort Worth Avenue,21051,1030 Fort Worth Avenue,Dallas,TX,US,75208,Dallas County,...,decimal,NaN,lov_entry,NaN,string,None,7930,"Oden Hughes Taylor Construction, LLC.",NaN,NaN
1,995936,10300 Metropolitan,19038 - 10300 Metropolitan,19038,10300 Metropolitan Drive,Austin,TX,US,78759,None,...,decimal,NaN,lov_entry,NaN,string,None,7930,"Oden Hughes Taylor Construction, LLC.",NaN,NaN
2,1433517,1210 W 5th St,20079 - 1210 W 5th St,20079,1210 W 5th Street,Austin,TX,US,78703,None,...,decimal,NaN,lov_entry,NaN,string,None,7930,"Oden Hughes Taylor Construction, LLC.",NaN,NaN
3,1792896,1604 O'Conner Townhomes,21058 - 1604 O'Conner Townhomes,21058,None,None,None,US,None,None,...,decimal,NaN,lov_entry,NaN,string,None,7930,"Oden Hughes Taylor Construction, LLC.",NaN,NaN
4,1771335,200 West John Carpenter,21056 - 200 West John Carpenter,21056,,Las Colinas,,US,,,...,decimal,NaN,lov_entry,NaN,string,None,7930,"Oden Hughes Taylor Construction, LLC.",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,1757867,Wolf Ranch,21054 - Wolf Ranch,21054,None,None,None,US,None,None,...,decimal,NaN,lov_entry,NaN,string,None,7930,"Oden Hughes Taylor Construction, LLC.",NaN,NaN
307,1666165,Woodland Green,19047 - Woodland Green,19047,,Austin,,US,,Travis County,...,decimal,NaN,lov_entry,NaN,string,None,7930,"Oden Hughes Taylor Construction, LLC.",NaN,NaN
308,917489,Workforce Concept,19026 - Workforce Concept,19026,,Austin,TX,US,,None,...,decimal,NaN,lov_entry,NaN,string,None,7930,"Oden Hughes Taylor Construction, LLC.",NaN,NaN
309,991172,Wrap Tower Ft Worth,19036 - Wrap Tower Ft Worth,19036,,Fort Worth,TX,US,,None,...,decimal,NaN,lov_entry,NaN,string,None,7930,"Oden Hughes Taylor Construction, LLC.",NaN,NaN


In [23]:
projects_df.columns

Index(['id', 'name', 'display_name', 'project_number', 'address', 'city',
       'state_code', 'country_code', 'zip', 'county', 'time_zone', 'latitude',
       'longitude', 'stage', 'phone', 'created_at', 'updated_at', 'active',
       'origin_id', 'origin_data', 'origin_code', 'owners_project_id',
       'estimated_value', 'project_region_id', 'project_bid_type_id',
       'project_owner_type_id', 'photo_id', 'start_date', 'completion_date',
       'total_value', 'accounting_project_number', 'store_number',
       'designated_market_area', 'custom_fields.custom_field_168942.data_type',
       'custom_fields.custom_field_168942.value',
       'custom_fields.custom_field_168943.data_type',
       'custom_fields.custom_field_168943.value',
       'custom_fields.custom_field_168944.data_type',
       'custom_fields.custom_field_168944.value',
       'custom_fields.custom_field_234689.data_type',
       'custom_fields.custom_field_234689.value',
       'custom_fields.custom_field_234690.da

In [24]:
projects_df['project_region_id']

0      245
1      243
2      243
3      244
4      245
      ... 
306    243
307    243
308    243
309    245
310    243
Name: project_region_id, Length: 311, dtype: int64

In [25]:
projects_df.city.unique()

array(['Dallas', 'Austin', None, 'Las Colinas', 'McKinney', 'San Marcos',
       'San Antonio', 'Irving', 'Friendswood', 'Webster',
       'Dripping Springs', 'Houston', 'Fort Worth ', 'Carrollton',
       'Celina', '', 'Cibolo', 'Conroe', 'Fort Worth', 'College Station',
       'Kyle', 'Cedar Park', 'Spring', 'Katy', 'Garland', 'Manchaca',
       'Round Rock', 'Fredricksburg', 'Georgetown', 'Denton',
       'Grand Prairie', 'Pflugerville', 'Richardson', 'Kemah', 'Kingwood',
       'La Porte', 'Lake Charles', 'Atascocita', 'Lakeway', 'Prosper',
       'Arlington', 'Garland ', 'Dallas ', 'Mansfield', 'Manor',
       'Leander', 'Euless', 'Rowlett', 'Sachse', 'Fairview', 'Cypress',
       'Nashville', 'austin', 'Magnolia', 'Sugarland', 'Rockwall',
       'Austin ', 'Humble', 'Plano', 'New Braunfels', 'The Woodlands',
       'Richmond', 'Anna'], dtype=object)

In [26]:
projects_df[["project_region_id"]]

,project_region_id
0,245
1,243
2,243
3,244
4,245
...,...
306,243
307,243
308,243
309,245


In [27]:
projects_df["custom_fields.custom_field_168942.value"].fillna(0, inplace=True)

projects_df["custom_fields.custom_field_234690.value"].fillna(0, inplace=True)

projects_df["custom_fields.custom_field_234689.value"].fillna(0, inplace=True)

projects_df["custom_fields.custom_field_241562.value"].fillna(0, inplace=True)

projects_df["address"].fillna(0, inplace=True)

projects_df["soiltype"].fillna(0, inplace=True)

# Iterate through DF and execute commands to add data to table
projects_df['company.id'] = projects_df['company.id'].replace(7930, '5058709')

projects_df["custom_fields.custom_field_168942.value"]

0      25th
1          
2          
3         0
4          
       ... 
306       0
307        
308        
309        
310    10th
Name: custom_fields.custom_field_168942.value, Length: 311, dtype: object

In [28]:
export_df = projects_df[['company.id', 'address', 'city', 'state_code', 'id', 'project_region_id', 'name', 'custom_fields.custom_field_234690.value', 'custom_fields.custom_field_234689.value', 'custom_fields.custom_field_241562.value', 'start_date']].copy()

In [34]:
conn = psycopg2.connect(
   database="Master_DB", user='OdenAdmin', password= "aJDbULW45C93zCykCxAp", host='oht-rds.cbunnafql9ym.us-east-2.rds.amazonaws.com', port= '5432'
)

#Setting auto commit false
conn.autocommit = True

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

# Define SQL execution command
insert_into_table = ("""INSERT INTO "OHT_projectinfo" ("CompanyID", "ProjectAddress", "ProjectCity", "ProjectState", "ProjectID", "ProjectRegion", "ProjectName", "Acres", "SoilType", "GroundWater", "EstStartDate") VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""")

# Iterate through DF and execute commands to add data to table
for i, row in export_df.iterrows():
   row_to_insert = (row['company.id'], row['address'], row['city'], row['state_code'], row['id'], row['project_region_id'],row['name'], row['custom_fields.custom_field_234690.value'], row['custom_fields.custom_field_234689.value'],row['custom_fields.custom_field_241562.value'], row['start_date'])
   cursor.execute(insert_into_table, row_to_insert)

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()

In [ ]:
get_codelist = "https://api.procore.com/rest/v1.0/standard_cost_code_lists?company_id=7930"

codeList = requests.get(get_codelist, headers={"Authorization": "Bearer " + token}).json()

print(codeList)

[{'id': 8237, 'is_erp_list': False, 'name': 'Standard Cost Code List'}]


In [ ]:
get_costcodes = "https://api.procore.com/rest/v1.0/standard_cost_codes?company_id=7930&standard_cost_code_list_id=8237"

costcodes = requests.get(get_costcodes, headers={"Authorization": "Bearer " + token}).json()

In [ ]:
df = pd.json_normalize(costcodes)

In [ ]:
df

,id,standard_cost_code_list_id,origin_id,parent_id,code,full_code,name,origin_data
0,7494613,8237,None,NaN,01-01,01-01,General Conditions,None
1,7494655,8237,None,7494613.0,100,01-01-100,Administrative Overhead,None
2,7494656,8237,None,7494613.0,110,01-01-110,Project Manager Payroll,None
3,7494657,8237,None,7494613.0,112,01-01-112,Project Coordinator Payroll,None
4,7494658,8237,None,7494613.0,120,01-01-120,Superintendent Payroll,None
...,...,...,...,...,...,...,...,...
247,7494851,8237,None,7494654.0,110,50-01-110,GC Fee,None
248,7494852,8237,None,7494654.0,200,50-01-200,Performance Bond,None
249,12738285,8237,None,7494654.0,300,50-01-300,Estimated Tax Credit,None
250,8938536,8237,None,NaN,55-55,55-55,Owner Changes,None
